<a href="https://colab.research.google.com/github/mandar33/BI-LSTM-CRF-FC/blob/main/Transform_i2b2_CoNLL_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import os
import re
import glob
import shutil
import requests
import gensim
import random



In [60]:
def countNumberOfFiles( path ,fileType):
    count = 0
    for c in glob.iglob(path+'/*.'+fileType):
        count+=1

    # print "The number of txt files in the directory is(Glob)", count
    return count

In [61]:
def countEntities( path,wordlist):

    # Open a file
    c_test=0
    c_problem= 0
    c_treatment = 0
    totalentitiesCount = 0

    for filename in glob.iglob(path+'/*.con'):
        # print(filename)
        f = open(filename,"r")
        for line in f:
            # print line
            tag =  str(line).split("||")
            entity =  tag[1].split("=")
            # print entity

            entity[1]= entity[1].replace("\"","")
            entity[1]= entity[1].replace("\n","")
            # print entity[1]

            if str(wordlist[0]) == str(entity[1])  :
                c_test+= 1
            if (wordlist[1] == entity[1])  :
               c_problem+= 1
            if (wordlist[2] == entity[1])  :
               c_treatment+= 1



        # print [c_test ,c_problem , c_treatment,totalentitiesCount]
    return [c_test ,c_problem , c_treatment]

In [62]:
def iob2(tags):
    """
    Check that tags have a valid IOB format.
    Tags in IOB1 format are converted to IOB2.
    """
    for i, tag in enumerate(tags):
        if tag == 'O':
            continue
        split = tag.split('-')
        if len(split) != 2 or split[0] not in ['I', 'B']:
            return False
        if split[0] == 'B':
            continue
        elif i == 0 or tags[i - 1] == 'O':  # conversion IOB1 to IOB2
            tags[i] = 'B' + tag[1:]
        elif tags[i - 1][1:] == tag[1:]:
            continue
        else:  # conversion IOB1 to IOB2
            tags[i] = 'B' + tag[1:]
    return True

In [67]:
train_sent_Dict = {}
train_sentLabels_Dict = {}

# The test dict contain the sentences and the corresponding labels used in evaluation of the model
test_sent_Dict = {}
test_sentLabels_Dict = {}

# These dicts would contain the line number and the sentences that is further used to split it into train and validation set
# Sentence Length to be considered for training and testing
#
SENTENCE_LENGTH = 1
ignored_lineCounterList = []
ignored_train_sent_Dict = {}
ignored_train_sentLabels_Dict = {}

# The test dict contain the sentences and the corresponding labels used in evaluation of the model
ignored_test_sent_Dict = {}
ignored_test_sentLabels_Dict = {}

def iob_iobes(tags):
    """
    IOB -> IOBES
    """
    new_tags = []
    for i, tag in enumerate(tags):
        if tag == 'O':
            new_tags.append(tag)
        elif tag.split('-')[0] == 'B':
            if i + 1 != len(tags) and \
               tags[i + 1].split('-')[0] == 'I':
                new_tags.append(tag)
            else:
                new_tags.append(tag.replace('B-', 'S-'))
        elif tag.split('-')[0] == 'I':
            if i + 1 < len(tags) and \
                    tags[i + 1].split('-')[0] == 'I':
                new_tags.append(tag)
            else:
                new_tags.append(tag.replace('I-', 'E-'))
        else:
            raise Exception('Invalid IOB format!')
    return new_tags


def iobes_iob(tags):
    """
    IOBES -> IOB
    """
    new_tags = []
    for i, tag in enumerate(tags):
        if tag.split('-')[0] == 'B':
            new_tags.append(tag)
        elif tag.split('-')[0] == 'I':
            new_tags.append(tag)
        elif tag.split('-')[0] == 'S':
            new_tags.append(tag.replace('S-', 'B-'))
        elif tag.split('-')[0] == 'E':
            new_tags.append(tag.replace('E-', 'I-'))
        elif tag.split('-')[0] == 'O':
            new_tags.append(tag)
        else:
            raise Exception('Invalid format!')
    return new_tags


def insert_singletons(words, singletons, p=0.5):
    """
    Replace singletons by the unknown word with a probability p.
    """
    new_words = []
    for word in words:
        if word in singletons and np.random.uniform() < p:
            new_words.append(0)
        else:
            new_words.append(word)
    return new_words


def computeConceptDict(conFilePath):
    cf = open(conFilePath,"r")
    cf_Lines = cf.readlines()
    line_dict = dict()


    for cf_line in cf_Lines:
        # print cf_line
        #c="a workup" 27:2 27:3||t="test"
        concept= cf_line.split("||")

        iob_wordIdx = concept[0].split()
        # print concept[0]
        iob_class = concept[1].split("=")
        iob_class = iob_class[1].replace("\"","")
        iob_class = iob_class.replace("\n","")

        # print iob_wordIdx[len(iob_wordIdx)-2],iob_wordIdx[len(iob_wordIdx)-1]
        start_iobLineNo = iob_wordIdx[len(iob_wordIdx)-2].split(":")
        end_iobLineNo = iob_wordIdx[len(iob_wordIdx)-1].split(":")
        start_idx = start_iobLineNo[1]
        end_idx = end_iobLineNo[1]
        iobLineNo=start_iobLineNo[0]
        # print "start",start_idx
        # print "end",end_idx

        # print "line Number, start_idx,end_idx, iobclass",iobLineNo,start_idx,end_idx,iob_class
        # line_dict.update({iobLineNo:start_idx+"-"+end_idx+"-"+iob_class})

        if iobLineNo in line_dict.keys():
                # append the new number to the existing array at this slot
                # print "Found duplicate line number....."
                line_dict[iobLineNo].append(start_idx+"-"+end_idx+"-"+iob_class)
        else:
                # create a new array in this slot
                line_dict.update({iobLineNo:[start_idx+"-"+end_idx+"-"+iob_class]})

    #
    # for k,v in line_dict.iteritems():
    #     print k,v

    return line_dict


def prepareIOB_wordList(wordList,lineNumber,IOBwordList,conceptDict,dataType):
    # print "Line Number",lineNumber
    # print "Word- List ",wordList

    iobTagList= []


    if str(lineNumber) in conceptDict.keys():
         # print conceptDict[str(lineNumber)]

         # split the tag and get the index of word and tag
         for concept in conceptDict[str(lineNumber)]:
             concept = str(concept).split("-")
             # print "start_idx, end_idx",concept[0],concept[1]
             # if (start_idx - end_idx) is zero then only B- prefix is applicable
             getrange = list(range(int(concept[0]),int(concept[1])))
             getrange.append(int(concept[1]))
             # For all the idx not in getrange assign an O tag
             # print getrange


             if(len(getrange) > 1):

                     for idx in range(0,len(getrange)):
                         # print getrange[idx]
                         iobTagList.append(int(getrange[idx]))
                         if(idx == 0):
                                IOBwordList[getrange[idx]] = "B-"+concept[2]
                         else:
                                 IOBwordList[getrange[idx]] = "I-"+concept[2]

             else:

                     idx = getrange[0]
                     iobTagList.append(int(getrange[0]))
                     # print idx
                     IOBwordList[idx] = "B-"+concept[2]



             # Else for all the indices between start and end apply the I- prefix

            # For all the other words assign O tag
         for i in range(0,len(IOBwordList)):
              if i not in iobTagList:
                IOBwordList[i] = "O"
         # print "IOB- WordList ",IOBwordList
    else:
         # print ""
         for i in range(0,len(IOBwordList)):
              if i not in iobTagList:
                IOBwordList[i] = "O"
         # print "IOB-  List ",IOBwordList
         # print "These Lines have ZERO IOB tags",IOBwordList
         # print "IOB Tag list ",iobTagList




    return IOBwordList




def createCONLL2003Data(inpath, outpath,dataType):
    
    files = os.listdir(cwd)  # Get all the files in that directory
    print(files)
    print("virat")
    # Make sure you have deleted all the old output files
    txtPath = inpath+"txt"
    conPath = inpath+"concept"
    print(os.listdir(txtPath))
    print(os.listdir(conPath))
    # remove all the *.txt files present in the path and rewrite it
    removefilesInDirectoryPath(outpath)

    if(dataType == "train"):
        train_Outfile = open(outpath+"train.txt", "a")
    else:
        test_Outfile = open(outpath+"test.txt", "a")

    conllfileContent=""
    filecounter = 0
    linecounter = 0
    
    # get all the list of file names only into the filenames list
    filenamesList = [os.path.basename(x) for x in glob.glob(txtPath+'/*.txt')]
    print('haaa')
    print(filenamesList)
    print('mandar')
    for filename in filenamesList:
        print(filename)
        filecounter+=1
        # print"The number of files processed are ",filecounter,(filename)
        f = open(txtPath+"/"+filename,'r')
        lines = f.readlines()
        # print "Number of Lines are : " ,len(lines)
        confileName=filename.split(".")
        confileName = confileName[0]
        print(confileName)
        conceptDict = computeConceptDict(conPath+"/"+confileName+".con")
        print(conceptDict)
        # print lines
        for line in range(0 ,len(lines)):
            words =  str(lines[line]).split()
            orginial_wordsList =  str(lines[line]).split()
            linecounter+=1

            IOBwordList= words
            # print words
            lineNumber= line+1 # Line number starts with 1


            #Prepare the IOB word list
            IOBwordList=prepareIOB_wordList(words,lineNumber,IOBwordList,conceptDict,dataType)

            # Merge the words and IOB words list in conll-2003 format

            for w in range(0,len(words)):
                conllfileContent= orginial_wordsList[w] + "\t" + IOBwordList[w] +"\n"
                # print conllfileContent
                if(dataType == "train"):
                    train_Outfile.write(conllfileContent)

                elif(dataType == "test"):
                    test_Outfile.write(conllfileContent)

             # add an Empty Line after each sentence conll 2003 format
            if(dataType == "train"):
                train_Outfile.write("\n")
            else:
                 test_Outfile.write("\n")

            if(dataType == "train"):
                   if(len(orginial_wordsList)== SENTENCE_LENGTH):
                        orginial_wordsList.append("PAD-WORD")
                        IOBwordList.append("O")

                   # print "The total number of sentences added untill now",linecounter,orginial_wordsList,IOBwordList
                   train_sent_Dict.update({linecounter:orginial_wordsList})
                   train_sentLabels_Dict.update({linecounter:IOBwordList})

            else:
                   if(len(orginial_wordsList)== SENTENCE_LENGTH):
                         orginial_wordsList.append("PAD-WORD")
                         IOBwordList.append("O")
                   test_sent_Dict.update({linecounter:orginial_wordsList})
                   test_sentLabels_Dict.update({linecounter:IOBwordList})


    return linecounter

def divideDataIntotrain_valid(inputDict):


    train_count= int(round(0.7 * len(inputDict)))
    valid_count= len(inputDict)- int(round(0.7 * len(inputDict)))
    return [train_count,valid_count]


import random

def createTrain_Valid_Set(maxrange,countTrain,countValid,ignoredLineCounterList):
    #generate a random number between 1 and 6502 and keep adding into list of index of sentences untill train_count
    count = 0
    train_sentList = []
    valid_sentList= []
  
    original_random_train_list = list(set([]))
    # print "Max Range",maxrange

    for n in range(1,maxrange+1,1):

        original_random_train_list.append(n)

    # print "Original Random List",original_random_train_list
    # print "Ignored sentences are ",ignoredLineCounterList


    # Remove all the ignored keys so that it does not throw  key not found exception in train and validation datasets

    # original_random_train_list = list(set(original_random_train_list)^set(ignoredLineCounterList))

    original_random_train_list= list(set(original_random_train_list).difference(ignoredLineCounterList))

    # print "After Removed Ignored Keys  ",original_random_train_list



    random.shuffle(original_random_train_list)

    # print " After Shuffling the original list",original_random_train_list


    # Split the list into train and valid size
    for e in original_random_train_list:
        if(count <= countTrain):
            train_sentList.append(e)
            count= count +1
        else:
           valid_sentList.append(e)

    # print("Max of random set")
    # print max(original_random_train_list)
    return [train_sentList,valid_sentList]


def removefilesInDirectoryPath(dirPath):
    # Change the directory to the current working directory
    os.chdir(dirPath)
    filelist = [ f for f in os.listdir(dirPath) if f.endswith(".txt") ]
    for f in filelist:
        os.remove(f)

    return



def save_train_validDatasets(train_sent_Dict,train_sentLabels_Dict,i2b2_train_OUTPATH,i2b2_valid_OUTPATH,train_sentList,valid_sentList):

     # First remove all the files in the directory
     removefilesInDirectoryPath(i2b2_train_OUTPATH)
     removefilesInDirectoryPath(i2b2_valid_OUTPATH)
     # Open train and validation file in the specified path
     train_Outfile = open(i2b2_train_OUTPATH+"/train.txt", "a")
     valid_Outfile = open(i2b2_valid_OUTPATH+"/dev.txt", "a")

     for sent_id in train_sentList:
         orginial_wordsList = train_sent_Dict[sent_id]
         IOBwordList = train_sentLabels_Dict[sent_id]
         for w in range(0,len(orginial_wordsList)):
                    conllfileContent= orginial_wordsList[w] + "\t" + IOBwordList[w] +"\n"
                    train_Outfile.write(conllfileContent)
         train_Outfile.write("\n") # after the end of every sentence write an new- line character


     for sent_id in valid_sentList:
         orginial_wordsList = train_sent_Dict[sent_id]
         IOBwordList = train_sentLabels_Dict[sent_id]
         for w in range(0,len(orginial_wordsList)):
                    conllfileContent= orginial_wordsList[w] + "\t" + IOBwordList[w] +"\n"
                    valid_Outfile.write(conllfileContent)
         valid_Outfile.write("\n") # after the end of every sentence write an new- line character # add an Empty Line after each sentence conll 2003 format


     return





def create_validationDataSet(train_sent_Dict,train_sentLabels_Dict,i2b2_train_OUTPATH,i2b2_valid_OUTPATH):

    [countTrain,countValid] = divideDataIntotrain_valid(train_sent_Dict)
    [train_sentList,valid_sentList]=createTrain_Valid_Set(len(train_sent_Dict),countTrain,countValid,ignored_lineCounterList)

    # print "training List of sentences are ",train_sentList
    # print "Validation list of sentences are ",valid_sentList

    print(" The number of training sentences are ",countTrain)
    print(" The number of validation sentences are ",countValid)
    print(" The total of training and test is ",countTrain+countValid)

    save_train_validDatasets(train_sent_Dict,train_sentLabels_Dict,i2b2_train_OUTPATH,i2b2_valid_OUTPATH,train_sentList,valid_sentList)

    return




In [21]:
import os
os.chdir('/content/')
cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

Files in '/content': ['.config', 'input_data', 'tagger', '.ipynb_checkpoints', 'output_data', 'sample_data']


In [74]:

i2b2_train_PATH = "/content/input_data/train/"
i2b2_test_PATH = "/content/input_data/test/"

i2b2_train_TEMP_OUTPATH = "/content/output_data/temp/"
i2b2_ignored_SENT_PATH = "/content/output_data/temp/ignored_sentences/"
i2b2_train_OUTPATH = "/content/output_data/train/"
i2b2_valid_OUTPATH = "/content/output_data/valid/"
i2b2_test_OUTPATH = "/content/output_data/test/"

i2b2CONLL2003PATH = "/content/tagger/data/conll2003/"
# Testing the concept file
i2b2_concept_testPATH = "/content/input_data/train/concept-test/018636330_DH.con"


#Remove previous train , valid and test files before creating train, test, validation files
train_totalSentences = createCONLL2003Data(i2b2_train_PATH, i2b2_train_OUTPATH,"train")
test_totalSentences=createCONLL2003Data(i2b2_test_PATH, i2b2_test_OUTPATH,"test")
print("Total Number of training  sentences",train_totalSentences)
print("Total Number of test  sentences",test_totalSentences)


# Combine the training and test sentences to create the vocabulary dict
vocab = set()
# append all the words from training sentences
for k, v in train_sent_Dict.items():
    vocab.update( sorted(set(w for w in v)))
# Append all the words from the test sentences
for k, v in test_sent_Dict.items():
    vocab.update( sorted(set(w for w in v)))
print(" The Number of words in Vocabulary " ,len(vocab)+1)

['.config', 'input_data', 'tagger', '.ipynb_checkpoints', 'output_data', 'sample_data']
virat
['record-36.txt', 'record-47.txt', 'record-84.txt', 'record-142.txt', 'record-65.txt', 'record-33.txt', 'record-69.txt', 'record-35.txt', 'record-74.txt', 'record-31.txt', 'record-18.txt', 'record-28.txt', 'record-52.txt', 'record-70.txt', 'record-15.txt', 'record-53.txt', 'record-141.txt', 'record-32.txt', 'record-49.txt', 'record-67.txt', 'record-23.txt', 'record-107.txt', 'record-108.txt', 'record-25.txt', 'record-16.txt', 'record-71.txt', 'record-45.txt', 'record-122.txt', 'record-55.txt', 'record-48.txt', 'record-83.txt', 'record-56.txt', 'record-22.txt', 'record-66.txt', 'record-26.txt', 'record-30.txt', '018636330_DH.txt', 'record-37.txt', 'record-81.txt', 'record-123.txt', 'record-82.txt', 'record-106.txt', 'record-13.txt', 'record-124.txt', 'record-51.txt', 'record-109.txt', 'record-105.txt', 'record-19.txt', 'record-73.txt', 'record-27.txt', 'record-17.txt', 'record-59.txt', 'record-

In [71]:
i2b2_train_PATH = "input_data/train/"
i2b2_test_PATH = "input_data/test/"

i2b2_train_TEMP_OUTPATH = "output_data/temp/"
i2b2_ignored_SENT_PATH = "output_data/temp/ignored_sentences/"
i2b2_train_OUTPATH = "output_data/train/"
i2b2_valid_OUTPATH = "output_data/valid/"
i2b2_test_OUTPATH = "output_data/test/"

i2b2CONLL2003PATH = "tagger/data/conll2003/"
# Testing the concept file
i2b2_concept_testPATH = "input_data/train/concept/018636330_DH.con"


In [75]:
noOftrain_Files = countNumberOfFiles(i2b2_train_PATH+"/txt","txt")
noOftrain_Concepts= countNumberOfFiles(i2b2_train_PATH+"/concept","con")
print("#(Training Files, Concept Files)", noOftrain_Files,noOftrain_Concepts)
#print(" Acutal no of files Reported files are ","349, 349")

noOftest_Files = countNumberOfFiles(i2b2_test_PATH+"/txt","txt")
noOftest_Concepts= countNumberOfFiles(i2b2_test_PATH+"/concept","con")
print("#(Test Files, Concept Files)", noOftest_Files,noOftest_Concepts)
#print(" Acutal no of files Reported files are ","477, 477")

# Code to count the number of entities present in training and test to be reported in basic statistics table
# The code counts the individual entities as well as total
# the types of entities present are problem, treatment, test
wordlist = ["test","problem","treatment"]
[c_test1 ,c_problem1 , c_treatment1] = countEntities(i2b2_train_PATH+"/concept/",wordlist)
totalentitiesCount=  c_test1 + c_problem1 + c_treatment1
print("Training Entities.. [c_test ,c_problem , c_treatment,totalentitiesCount]", [c_test1 ,c_problem1 , c_treatment1,totalentitiesCount])

[c_test2 ,c_problem2 , c_treatment2] = countEntities(i2b2_test_PATH+"/concept",wordlist)
totalentitiesCount=  c_test2 + c_problem2 + c_treatment2
print("Test Entities.. [c_test ,c_problem , c_treatment,totalentitiesCount]", [c_test2 ,c_problem2 , c_treatment2,totalentitiesCount])

#(Training Files, Concept Files) 66 66
#(Test Files, Concept Files) 8 8
Training Entities.. [c_test ,c_problem , c_treatment,totalentitiesCount] [2751, 3746, 2747, 9244]
Test Entities.. [c_test ,c_problem , c_treatment,totalentitiesCount] [361, 550, 404, 1315]
